<a href="https://colab.research.google.com/github/itsZENR/E-CUP/blob/main/E_CUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Надо по названиям, атрибутам и картинкам определить являются ли два товара одиноковыми

#1. Сравнение названий

    1. Обработка текста:
      Приведение названий и атрибутов к нижнему регистру.
      Удаление лишних пробелов, специальных символов и т.д.

    2. Семантическое сравнение:
      Использование методов обработки естественного языка (NLP) для вычисления семантической близости (например, с помощью библиотек таких как spaCy, nltk, или моделей как BERT).

In [7]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# https://habr.com/ru/companies/otus/articles/755772/

def preprocess_text(text):
    # Приведение текста к нижнему регистру и удаление специальных символов
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    # text = re.sub(r'[^\w\s]', '', text)
    return text

def compare_texts(text1, text2):
    text1 = preprocess_text(text1)
    text2 = preprocess_text(text2)

    # Использование TF-IDF для преобразования текста в векторное представление
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()

    # Вычисление косинусного сходства
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

text_similarity = compare_texts("Название товара 1", "Название товара eae4wdf 5wr")
print(f"Text Similarity: {text_similarity}")


Text Similarity: 0.5797386715376658


In [ ]:
# Тест
text = 'Строка!@ с# %специальными* знаками&'
print(preprocess_text(text))
text = 'Название товара 1'
print(preprocess_text(text))


строка с специальными знаками 
название товара 1


#2. Сравнение атрибутов

In [9]:
def compare_attributes(attr1, attr2):
    # Сравнение значений атрибутов
    matches = 0
    total = len(attr1)

    for key in attr1:
        if key in attr2 and attr1[key] == attr2[key]:
            matches += 1

    similarity = matches / total
    return similarity

attr1 = {"color": "red", "size": "M", "brand": "Nike"}
attr2 = {"color": "red", "size": "L", "brand": "Nike"}
attribute_similarity = compare_attributes(attr1, attr2)
print(f"Attribute Similarity: {attribute_similarity}")


Attribute Similarity: 0.6666666666666666


#3. Сравнение изображений

    1. Обработка изображений:
        Предобработка изображений (например, изменение размера, нормализация).

    2. Вычисление признаков изображений:
        Использование моделей глубокого обучения для извлечения признаков (например, с помощью ResNet, VGG или других CNN).

    3. Сравнение признаков:
        Вычисление расстояний между векторами признаков (например, косинусное расстояние, евклидово расстояние).

In [5]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import requests
from io import BytesIO
from PIL import Image


def preprocess_image(img):
    # Загрузка и предобработка изображения
    img = image.load_img(img, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data

def extract_features(img, model):
    img_data = preprocess_image(img)
    features = model.predict(img_data)
    return features.flatten()

def compare_images(img1, img2):
    base_model = VGG16(weights='imagenet')
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

    features1 = extract_features(img1, model)
    features2 = extract_features(img2, model)

    # Вычисление косинусного расстояния между признаками
    cosine_sim = cosine_similarity([features1], [features2])
    return cosine_sim[0][0]

def url_download(url: str):
    response = requests.get(url)
    img_data = response.content
    return BytesIO(img_data)


url1 = 'https://cf-kr.kcdn.online/content/4f/665660e2d43be4d5123kak-vybrat-dom-main-9601.jpg'
url2 = 'https://cf-kr.kcdn.online/content/4f/665660e2d43be4d5123kak-vybrat-dom-main-9601.jpg'
img1 = url_download(url1)
img2 = url_download(url2)
image_similarity = compare_images(img1, img2)
print(f"Image Similarity: {image_similarity}")


1/1 [==============================] - 1s 508ms/step
Image Similarity: 0.9999997615814209


#4. Комплексное сравнение

    1. Комбинирование результатов текстового и визуального сравнения для принятия окончательного решения.

In [13]:
def compare_products(text1, text2, img1, img2, attr1, attr2):
    text_similarity = compare_texts(text1, text2)
    attribute_similarity = compare_attributes(attr1, attr2)
    image_similarity = compare_images(img1, img2)


    # Комбинирование результатов
    overall_similarity = (text_similarity + attribute_similarity + image_similarity) / 3
    return overall_similarity

text1 = 'Название товара 1'
text2 = 'Название товара 2'
url1 = 'https://cf-kr.kcdn.online/content/4f/665660e2d43be4d5123kak-vybrat-dom-main-9601.jpg'
url2 = 'https://cf-kr.kcdn.online/content/4f/665660e2d43be4d5123kak-vybrat-dom-main-9601.jpg'
img1 = url_download(url1)
img2 = url_download(url2)
attr1 = {"color": "red", "size": "M", "brand": "Nike"}
attr2 = {"color": "red", "size": "L", "brand": "Nike"}
overall_similarity = compare_products(text1, text2, img1, img2, attr1, attr2)
if overall_similarity > 0.8:
  print('Товары похожи')
else:
  print('Товары не похожи')
print(overall_similarity)

1/1 [==============================] - 1s 504ms/step
Товары похожи
0.8888888094160293
